In [1]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from tqdm import tqdm
import time
import pandas as pd
import datetime
from bs4 import BeautifulSoup
import os

In [2]:
date_target = '01.12.2017'
cwd = os.getcwd()

In [49]:
def check_need_new_parse(file_name='parse_investing.xlsx', diff_day = 3):
    load_dir = os.getcwd()
    load_file = f'{load_dir}\parse_investing.xlsx'
    
    try:
        df_parse = pd.read_excel(load_file)
        print('Старый ДФ прочитан')
    except:
        print(f'Файл {file_name} не найден. Будет загружен в папку: ')
        print(load_dir)
        need_new_data = True
        return True
    
    
    date_old = pd.to_datetime(df_parse['Дата'], format='%d.%m.%Y').max()
    date_now_shift = datetime.datetime.today() - datetime.timedelta(days=diff_day)
    
    date_old_srt = date_old.strftime('%Y-%m-%d')
    
    if date_old > date_now_shift:
        print(f'Новые данные не парсятся. Cтарая дата: {date_old_srt}. Отстаёт от сегодя меньше чем на {diff_day} дней.')
        need_new_data = False
    else:
        print(f'Данные будут обновлены. Старая дата: {date_old_srt}. Отстаёт от сегодя больше чем на {diff_day} дней.')
        need_new_data = True
    return need_new_data


check_need_new_parse()

Старый ДФ прочитан
Новые данные не парсятся. Cтарая дата: 2023-05-21. Отстаёт от сегодя меньше чем на 3 дней.


False

In [3]:
url = 'https://ru.investing.com/crypto/bitcoin/btc-usd-historical-data?cid=1035793'
       

def create_driver(driver, url):
    chrome_options = Options()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), chrome_options=chrome_options)
    driver.get(url)
    time.sleep(3)
    
    return driver


def change_window(driver):
    # Увеличиваю окно браузера
    driver.set_window_size(1800, 1000)

    # Прокрутить
    driver.execute_script("window.scrollTo(0, 650);") ### https://qastack.ru/programming/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python


def change_date():
    # Открываю поле воода даты
    butom_date = 'DatePickerWrapper_input__MDvWH'
    driver.find_element(By.CLASS_NAME, butom_date).click()
    time.sleep(2)

    # Ввод даты старта.
    for try_n in range(1, 5):
        print(f'Попытка установки даты {try_n}')
        butom_close = 'NativeDateInput_root__wbgyP'  #'#__next > div > div > div > div.grid.gap-4.tablet\:gap-6.grid-cols-4.tablet\:grid-cols-8.desktop\:grid-cols-12.grid-container--fixed-desktop.general-layout_main__lRLYJ > main > div > div:nth-child(5) > div > div:nth-child(1) > div > div.flex.justify-between.items-end.mb-4 > div.flex.flex-row-reverse.desktop\:flex-row.gap-6 > div.historical-data_history-date-picker-wrapper__dDOuq > div.HistoryDatePicker_HistoryDatePicker__sjrlU > div:nth-child(1) > div > div:nth-child(1) > input[type=date]' #'//*[@id="__next"]/div/div/div/div[2]/main/div/div[5]/div/div[1]/div/div[2]/div[2]/div[2]/div[2]/div[1]/div/div[1]/input'


        field = driver.find_element(By.CLASS_NAME, butom_close) #CSS_SELECTOR
        field = field.find_element(By.TAG_NAME, 'input')
        # field.clear()
        field.send_keys(date_target) # date_target + Keys.ENTER
        time.sleep(2)
        # Дата окончани не нужна. 
        if field.get_attribute('value')[:4] == date_target[-4:]:
            print('Утановленая дата =',  field.get_attribute('value'))
            print('Успешно')
            break
        else:
            print('Утановленая дата =',  field.get_attribute('value'))

    # Применить фильтр.
    butom_ = 'HistoryDatePicker_arrow-icon__NwxN4'  #'//*[@id="__next"]/div/div/div/div[2]/main/div/div[5]/div/div[1]/div/div[2]/div[2]/div[2]/div[2]/div[2]/button'
    driver.find_element(By.CLASS_NAME, butom_).click()
    time.sleep(2)

    # Проверить установился ли фильтр даты.
    butom_ ='historical-data_history-date-picker-wrapper__dDOuq'
    date_text = driver.find_element(By.CLASS_NAME, butom_).text

    if date_text[:len(date_target)][-4:] == date_target[-4:]:
        print('Дата установлена верно')
    else:
        print('Утановленная дата', date_text[:len(date_target)])
        1/0


# Прокрутить
driver.execute_script("window.scrollTo(0, 900);")

col_names = ['Дата', 'Цена', 'Откр.', 'Макс.', 'Мин.', 'Объём', 'Изм. %']


soup = BeautifulSoup(driver.page_source, 'lxml')
# https://medium.com/analytics-vidhya/scraping-tables-from-a-javascript-webpage-using-selenium-beautifulsoup-and-pandas-cbd305ca75fe
# beautifulsoup4 используемая библиотека

tables = soup.find_all('table')
dfs = pd.read_html(str(tables))
for df_i in dfs:
    head_name = list(df_i)
    if head_name == ['Дата', 'Цена', 'Откр.', 'Макс.', 'Мин.', 'Объём', 'Изм. %']:
        df_parse = df_i.copy()
        break

[WDM] - Downloading:  99%|████████████████████████████████████████████████████████▋| 6.77M/6.81M [00:14<00:00, 509kB/s]<ipython-input-3-fcef8dd5f9b1>:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), chrome_options=chrome_options)
[WDM] - Downloading: 100%|█████████████████████████████████████████████████████████| 6.81M/6.81M [00:30<00:00, 509kB/s]

Попытка установки даты 1
Утановленая дата = 0007-11-29
Попытка установки даты 2
Утановленая дата = 2017-11-28
Успешно


[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.81M/6.81M [02:05<00:00, 56.8kB/s]

Дата установлена верно


In [4]:
df_parse['Дата'] = pd.to_datetime(df_parse['Дата'], format='%d.%m.%Y')

In [5]:
def replace_num(str_):
    str_ = str(str_)
    if 'K' in str_:
        return int(str_.replace('K', '').replace(',', '')+'000')
    elif 'M' in str_:
        return int(str_.replace('M', '').replace(',', '')+'000000')
    else:
        try:
            return int(str_.replace(',', ''))
        except:
            return 0
        
df_parse['value_correct'] = df_parse['Объём'].apply(replace_num)

In [6]:
def replace_symb(str_):
    str_ = float(str(str_).replace('.', '').replace(',', '.'))
    return str_

In [7]:
for col in ['Цена', 'Откр.', 'Макс.', 'Мин.', 'value_correct']:
    print(col)
    df_parse[col] = df_parse[col].apply(replace_symb)


driver.close()

Цена
Откр.
Макс.
Мин.
value_correct


In [8]:
df_parse.to_excel('parse_investing.xlsx', index=False)

# Parce google trands

In [31]:
url = '''https://trends.google.ru/trends/?geo=KZ'''

chrome_options = Options()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), chrome_options=chrome_options)
driver.get(url)

# Увеличиваю окно браузера
driver.set_window_size(1800, 1000)

<ipython-input-31-b107116fa416>:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), chrome_options=chrome_options)


In [32]:
# # Ввод поискового запроса
butom_close = '//*[@id="input-254"]'
find_data = 'buy bitcoin' 

field = driver.find_element(By.XPATH, butom_close)
field.clear()
field.send_keys(find_data + Keys.ENTER) # date_target + Keys.ENTER
time.sleep(2)
# Дата окончани не нужна. 

In [33]:
# Открываю список регионов
butom_ = '//*[@id="compare-pickers-wrapper"]/div/hierarchy-picker[1]/ng-include/div[1]'
find_data = 'buy bitcoin' 
set_local = 'По всему миру'
set_local = 'Казахстан'


field = driver.find_element(By.XPATH, butom_).click()
time.sleep(1)


In [34]:
# Ввожу что мне нужны данные по всему миру
butom_ = '//*[@id="input-8"]'
set_local = 'По всему миру'


field = driver.find_element(By.XPATH, butom_)
field.clear() #clear
field.send_keys(set_local + Keys.ENTER)
time.sleep(3)


In [35]:
# Выбираю из выподающего списка
butom_ = '//*[@id="ul-8"]/li/md-autocomplete-parent-scope/div'

find_data = 'buy bitcoin' 
set_local = 'По всему миру'


field = driver.find_element(By.XPATH, butom_).click()
time.sleep(3)


In [36]:
# Открываю список периодов
butom_ = 'compare-picker ng-pristine ng-untouched ng-valid ng-not-empty'.replace(' ', '.')

field = driver.find_element(By.CLASS_NAME, butom_).click()
time.sleep(3)

# Поиск по нескольким классам.
# https://stackoverflow.com/questions/60534244/how-to-locate-an-element-with-multiple-classnames-using-selenium-and-python

In [37]:
# Нажимаю произвольный период.
butom_ = 'custom-date-picker-select-option md-ink-ripple'.replace(' ', '.')

driver.find_elements(By.CLASS_NAME, butom_)[-1].click()
time.sleep(2)

In [38]:
# Выбираю дату начала.
butom_ = 'md-datepicker-input'


field = driver.find_element(By.CLASS_NAME, butom_)
field.clear()
field.send_keys(date_target)
time.sleep(3)

In [39]:
# Нажимаю ок

n_d = 3 # number_decision

butom_ = 'custom-date-picker-dialog-button md-button md-ink-ripple'.replace(' ', '.')

field = driver.find_elements(By.CLASS_NAME, butom_)
field[-1].click()

time.sleep(3)

In [40]:
# Скачать данные

n_d = 1 # number_decision

butom_ = 'widget-actions-item export'.replace(' ', '.')

field = driver.find_elements(By.CLASS_NAME, butom_)[0].click()
time.sleep(3)

In [41]:
driver.close()

In [42]:
# Находим последний загруженнй файл
dowlend_path = 'C:/Users/Kirill/Downloads'
list_paths = []

for f_name in os.listdir(dowlend_path):
    dowlend_file_p = dowlend_path + '//' + f_name
    list_paths.append(dowlend_file_p)

latest_file = max(list_paths, key=os.path.getctime)
# Получить название последнего файла
# https://stackoverflow.com/questions/39327032/how-to-get-the-latest-file-in-a-folder

# Получить дату изменения файла
# https://stackoverflow.com/questions/237079/how-do-i-get-file-creation-and-modification-date-times

In [43]:
# Убираю файл из загрузок
df_google_trand = pd.read_csv(latest_file)
df_google_trand.to_excel('google_trand.xlsx')

os.remove(latest_file)

In [56]:
# Сохраняю файл в текущую папку.
save_name = cwd + "\\" + 'google_trand.xlsx'
df_google_trand[1:].to_excel(save_name, index=False)

'D:\\Analyst_profession\\PROGECT\\BTC Find Anamaly\\work git\\search_anomaly_Bitcoin\\google_trand.xlsx'